In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import random
import scipy.stats as spstats
%matplotlib inline

In [ ]:
def load_survey(round_number, filename):
    df = pd.read_csv(filename, delimiter='\t', encoding='utf-16').drop([0,1])[[
        'IPAddress', 'participant_id', 'group_id', 'session_code', 'Q2_1', 'Q2_2', 'Q2_3', 'Q2_4'
    ]]
    df['group_id'] = pd.to_numeric(df.group_id)
    df['participant_id'] = pd.to_numeric(df.participant_id)
    na = df.group_id.isna()
    df.loc[na, 'group_id'] = 100
    return df


In [ ]:
df_a = load_survey(0, 'results/2018-12-21/Networked-Deliberation-01-A_December 21, 2018_11.41.tsv')
df_b = load_survey(1, 'results/2018-12-21/Networked-Deliberation-01-B_December 21, 2018_11.42.tsv')
df_c = load_survey(2, 'results/2018-12-21/Networked-Deliberation-01-C_December 21, 2018_11.43.tsv')
df_d = load_survey(3, 'results/2018-12-21/Networked-Deliberation-01-D_December 21, 2018_11.44.tsv')

df = pd.concat([df_a, df_b, df_c, df_d]).rename({
    "Q2_1": "item1_rank",
    "Q2_2": "item2_rank",
    "Q2_3": "item3_rank",
    "Q2_4": "item4_rank",
}, axis=1)

exclude_ids = set([1, 4])
participant_ids = set(df.participant_id)
df = df[df.participant_id.isin(participant_ids - exclude_ids)]

In [ ]:
def ranked_to_ranks(alternatives):
    rank_alt = enumerate(alternatives)
    rank_alt = sorted(rank_alt, key=lambda x: x[1])
    ranks = [x[0] for x in rank_alt]
    return ranks

def get_preferences(df, exclude_id=[]):
    participant_ids = sorted(set(df.participant_id))
    preferences = {}
    for pid in participant_ids:
        if pid in exclude_id:
            continue
        participant_df = df[df.participant_id == pid].sort_values('group_id')
        preferences[pid] = []
        for i, group_id in enumerate(sorted(set(participant_df.group_id))):
            rank1 = list(participant_df.item1_rank)[i]
            rank2 = list(participant_df.item2_rank)[i]
            rank3 = list(participant_df.item3_rank)[i]
            rank4 = list(participant_df.item4_rank)[i]
            ranks = [(rank1, 1), (rank2, 2), (rank3, 3), (rank4, 4)]
            preference = [x[1] for x in sorted(ranks)]
            preferences[pid].append(preference)
    return preferences
   
preferences = get_preferences(df)
preferences

In [ ]:
def get_profile(df, exclude_id=[]):
    preferences = get_preferences(df)
    num_rounds = len(list(preferences.values())[0])
    participants = preferences.keys()
    profiles = []
    for n in range(num_rounds):
        profile = {}
        profiles.append(profile)
        for p in participants:
            pref = preferences[p][n]
            try:
                profile[tuple(pref)] += 1
            except KeyError:
                profile[tuple(pref)] = 1
    return profiles
    
profiles = get_profile(df, exclude_id=[1, 4])

In [ ]:
def print_preference_evolution(df):
    alternatives = sorted(set(sum([list(p.keys()) for p in profiles], [])))
    counts = []
    for alt in alternatives:
        line = [repr(alt)]
        for round_id in range(4):
            line.append(str(profiles[round_id].get(alt, '')))
        print('\t'.join(line))
print_preference_evolution(df)

In [ ]:
def print_spearman_evolution(df):
    alternatives = sorted(set(sum([list(p.keys()) for p in profiles], [])))
    counts = []
    N = sum(profiles[0].values())
    for pref_a in alternatives:
        line = [repr(pref_a)]
        for round_id in range(4):
            total = 0
            for pref_b in alternatives:
                if pref_a == pref_b:
                    n = profiles[round_id].get(pref_a, 0) * (profiles[round_id].get(pref_b, 0) - 1) / 2
                else:
                    n = profiles[round_id].get(pref_a, 0) * profiles[round_id].get(pref_b, 0)
                a = ranked_to_ranks(pref_a)
                b = ranked_to_ranks(pref_b)
                r, p = spstats.spearmanr(a, b)
                total += r * n
            if total == 0:
                line.append('')
            else:
                line.append('{:+0.2f}'.format(total / (N*(N-1)/2)))
        print('\t'.join(line))
print_spearman_evolution(df)

In [ ]:
get_preferences(df)

In [ ]:
def print_participant_spearman_evolution(df):
    preferences = get_preferences(df)
    profiles = get_profile(df)
    participants = sorted(set(df.participant_id))
    N = len(participants)
    for part in participants:
        line = [repr(part)]
        for round_id in range(4):
            pref_a = preferences[part][round_id]
            total = 0
            alternatives = profiles[round_id].keys()
            for pref_b in alternatives:
                n = profiles[round_id].get(pref_b, 0)
                if pref_a == pref_b:
                    n -= 1
                a = ranked_to_ranks(pref_a)
                b = ranked_to_ranks(pref_b)
                r, p = spstats.spearmanr(a, b)
                total += r * n
            if total == 0:
                line.append('')
            else:
                line.append('{:+0.2f}'.format(total / (N*(N-1)/2)))
        print('\t'.join(line))
print_participant_spearman_evolution(df)

In [ ]:
def score_majority(profile):
    votes = {}
    for pref, count in profile.items():
        top = pref[0]
        try:
            votes[top] += count
        except KeyError:
            votes[top] = count
    return votes

def social_preference_majority(profile):
    score = score_majority(profile)
    ordered_scores = sorted(score.items(), reverse=True, key=lambda x: x[1])
    social_preference = []
    last_score = ordered_scores[0][1]
    current_set = set()
    for alternative, score in ordered_scores:
        # If the score has changed, create a new set of alternatives
        if score == last_score:
            current_set.add(alternative)
        else:
            social_preference.append(current_set)
            current_set = set([alternative])
        last_score = score
    social_preference.append(current_set)
    return social_preference

def social_choice_majority(profile):
    social_preference = social_preference_majority(profile)
    return social_preference[0]

for p in profiles:
    print(social_choice_majority(p), social_preference_majority(p))

In [ ]:
def borda_count(profile):
    borda = {}
    for pref, count in profile.items():
        for i, alternative in enumerate(reversed(pref)):
            try:
                borda[alternative] += i * count
            except KeyError:
                borda[alternative] = i * count
    return borda

def social_preference_borda(profile):
    borda = borda_count(profile)
    ordered_scores = sorted(borda.items(), key=lambda x: x[1], reverse=True)
    social_preference = []
    last_score = ordered_scores[0][1]
    current_set = set()
    for alternative, score in ordered_scores:
        # If the score has changed, create a new set of alternatives
        if score == last_score:
            current_set.add(alternative)
        else:
            social_preference.append(current_set)
            current_set = set([alternative])
        last_score = score
    social_preference.append(current_set)
    return social_preference

def social_choice_borda(profile):
    return social_preference_borda(profile)[0]

for profile in profiles:
    print(social_choice_borda(profile), social_preference_borda(profile))

In [ ]:
import itertools
def score_tideman(profile):
    alternatives = set(sum([list(pref) for pref in profile.keys()], []))
    pairs = set(itertools.product(alternatives, alternatives))
    for alt in alternatives:
        pairs.remove((alt, alt))
    margins = dict((contest, 0) for contest in pairs)
    for pref, count in profile.items():
        for win in range(len(pref)):
            for lose in range(win + 1, len(pref)):
                margins[(pref[win], pref[lose])] += count
    return margins
        

import networkx as nx
def tideman_graph(profile):
    margins = score_tideman(profile)
    ordered_margins = sorted(margins.items(), key=lambda x: x[1], reverse=True)
    G = nx.DiGraph()
    counted = 0
    skipped = 0
    for contest, margin in ordered_margins:
        s, t = contest
        try:
            if nx.has_path(G, t, s):
                # Skip contest, would create cycle
                skipped += margin
                continue
        except nx.NodeNotFound:
            pass
        # Add node to graph
        G.add_edge(s, t)
        counted += margin
    return G, counted, skipped

def social_preference_tideman(profile):
    G, counted, skipped = tideman_graph(profile)
    root = next(nx.topological_sort(G))
    social_preference = []
    done = set()
    shell = set([root])
    while len(shell) > 0:
        next_shell = set()
        # Remove alternatives ranked lower than others in shell
        for s in list(shell):
            for t in list(shell):
                if G.has_edge(s, t):
                    shell.remove(t)
        # Update list of completed alternatives
        done = done | shell
        # Build next shell
        for s in shell:
            next_shell = (next_shell | set(G.successors(s))) - done
        # Add current shell to preference
        social_preference.append(shell)
        shell = next_shell
    return social_preference

def social_choice_tideman(profile):
    return social_preference_tideman(profile)[0]

def concurrence_tideman(profile):
    G, counted, skipped = tideman_graph(profile)
    return counted / (counted + skipped)

for p in profiles:
    print(social_preference_tideman(p))

## Group-level

In [ ]:
groups = sorted(set(df.group_id) - set([100.0]))
group_profile = {}

for g in groups:
    dfg = df[df.group_id == g]
    group_profile[g] = get_profile(dfg, exclude_id=[1, 4])

In [ ]:
group_majority = {}
for g in groups:
    group_majority[g] = [
        social_choice_majority(p) for p in group_profile[g]]
group_majority

In [ ]:
group_borda = {}
for g in groups:
    group_borda[g] = [
        social_choice_borda(p) for p in group_profile[g]]
group_borda

In [ ]:
group_tideman = {}
for g in groups:
    group_tideman[g] = [
        social_choice_tideman(p) for p in group_profile[g]]
group_tideman

## Concurrence

In [ ]:
def sample_to_profile(sample):
    profile = dict()
    # Walk through sorted list of samples
    # Walks through once total, faster than using count() many times
    ordered_sample = sorted(sample)
    current_alt = ordered_sample[0]
    current_count = 0
    for x in ordered_sample:
        if x != current_alt:
            profile[current_alt] = current_count
            current_count = 1
            current_alt = x
        else:
            current_count += 1
    profile[current_alt] = current_count
    return profile

In [ ]:
def bootstrap_profile(profile):
    population = sum([[alt] * i for alt, i in profile.items()], [])
    k = len(population)
    sample = [
        population[random.randint(0, k - 1)]
        for i in range(k)]
    profile = sample_to_profile(sample)
    return profile

In [ ]:
def concurrence_spearman(profile):
    total = 0
    count = 0
    for pref_a, count_a in profile.items():
        for pref_b, count_b in profile.items():
            if repr(pref_a) == repr(pref_b):
                continue
            a = ranked_to_ranks(pref_a)
            b = ranked_to_ranks(pref_b)
            r, p = spstats.spearmanr(a, b)
            total += count_a * count_b * r
            count += count_a * count_b
    return total / count

def concurrence_bootstrap(profile, f, N=1000, sigma=1.0):
    estimates = [
        f(bootstrap_profile(profile))
        for n in range(N)]
    error = np.std(estimates)
    return sigma * error

def concurrence_kendall(profile):
    total = 0
    count = 0
    for pref_a, count_a in profile.items():
        for pref_b, count_b in profile.items():
            if repr(pref_a) == repr(pref_b):
                continue
            a = ranked_to_ranks(pref_a)
            b = ranked_to_ranks(pref_b)
            tau, p = spstats.kendalltau(a, b)
            total += count_a * count_b * tau
            count += count_a * count_b
    return total / count


            

In [ ]:
spearman = []
for p in profiles:
    c = concurrence_spearman(p)
    c_err = concurrence_bootstrap(p, concurrence_spearman, N=1000)
    spearman.append(c)
    print('{:0.4f} +/- {:0.4f}'.format(c, c_err))

In [ ]:
kendall = []
for p in profiles:
    c = concurrence_kendall(p)
    c_err = concurrence_bootstrap(p, concurrence_kendall, N=1000)
    kendall.append(c)
    print('{:0.4f} +/- {:0.4f}'.format(c, c_err))

In [ ]:
tideman = []
for p in profiles:
    c = concurrence_tideman(p)
    c_err = concurrence_bootstrap(p, concurrence_tideman, N=1000)
    tideman.append(c)
    print('{:0.4f} +/- {:0.4f}'.format(c, c_err))


In [ ]:
plt.figure(figsize=(3,2.5))
plt.plot(spearman, label='Spearman')
plt.plot(kendall, label='Kendall')
plt.plot(tideman, label='Tideman')
plt.legend()
plt.xticks([0, 1, 2, 3])
plt.xlabel('Stage')
plt.ylabel('Concurrence')
plt.tight_layout()
plt.savefig('concurrence.png', dpi=1200)

## Group Level Analysis

In [ ]:
groups = [
    [13, 14, 15, 16],
    [17, 18, 19, 20],
    [21, 22, 23, 24]
]

In [ ]:
df['round_id'] = 0
df.loc[df.group_id == 100.0, 'round_id'] = 3
df.loc[df.group_id < 99, 'round_id'] = 2
df.loc[df.group_id < 21, 'round_id'] = 1
df.loc[df.group_id < 17, 'round_id'] = 0

In [ ]:
def get_group_profiles(df, group_id):
    # Construct data frame for group
    df_group = df[df.group_id == group_id]
    participant_ids = set(df_group.participant_id)
    round_id = list(df_group.round_id)[0]
    next_round_id = round_id + 1
    df_next = df[
        df.participant_id.isin(participant_ids)
        & (df.round_id == next_round_id)]
    
    profile = {}
    for pid in participant_ids:
        participant_df = df_group[df_group.participant_id == pid]
        rank1 = list(participant_df.item1_rank)[0]
        rank2 = list(participant_df.item2_rank)[0]
        rank3 = list(participant_df.item3_rank)[0]
        rank4 = list(participant_df.item4_rank)[0]
        ranks = [(rank1, 1), (rank2, 2), (rank3, 3), (rank4, 4)]
        preference = tuple([x[1] for x in sorted(ranks)])
        try:
            profile[preference] += 1
        except KeyError:
            profile[preference] = 1
    
    next_profile = {}
    for pid in participant_ids:
        participant_df = df_next[df_next.participant_id == pid]
        rank1 = list(participant_df.item1_rank)[0]
        rank2 = list(participant_df.item2_rank)[0]
        rank3 = list(participant_df.item3_rank)[0]
        rank4 = list(participant_df.item4_rank)[0]
        ranks = [(rank1, 1), (rank2, 2), (rank3, 3), (rank4, 4)]
        preference = tuple([x[1] for x in sorted(ranks)])
        try:
            next_profile[preference] += 1
        except KeyError:
            next_profile[preference] = 1
    return (profile, next_profile)


In [ ]:
print("Round\tGroup\tC\tC_next\tChange")
for round_id, round_groups in enumerate(groups):
    for g in round_groups:
        profile, next_profile = get_group_profiles(df, g)
        c = concurrence_spearman(profile)
        c_next = concurrence_spearman(next_profile)
        print(round_id, '\t', g, '\t', '{:+0.2f}\t{:+0.2f}\t{:+0.2f}'.format(c, c_next, c_next - c))

In [ ]:
df_group = df[df.group_id == 21]
participant_ids = set(df_group.participant_id)
round_id = list(df_group.round_id)[0]
next_round_id = round_id + 1
df_next = df[
    df.participant_id.isin(participant_ids)
    & (df.round_id == next_round_id)]


In [ ]:
df[df.participant_id == 5]

In [ ]:
df_next